In [34]:

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

In [37]:
data ='fra.txt'
len(data)

7

In [43]:

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text,_ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [44]:

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [45]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [46]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


In [47]:

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [48]:

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [49]:

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [50]:

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [51]:

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [53]:

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
             metrics = ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
# model.save('s2s.h5')

Epoch 1/100
125/125 [==============================] - 2s 20ms/step - loss: 0.0536 - accuracy: 0.2843 - val_loss: 0.7161 - val_accuracy: 0.2176
Epoch 2/100
125/125 [==============================] - 2s 13ms/step - loss: 0.0506 - accuracy: 0.2854 - val_loss: 0.7268 - val_accuracy: 0.2176
Epoch 3/100
125/125 [==============================] - 2s 13ms/step - loss: 0.0500 - accuracy: 0.2855 - val_loss: 0.7266 - val_accuracy: 0.2177
Epoch 4/100
125/125 [==============================] - 2s 13ms/step - loss: 0.0492 - accuracy: 0.2855 - val_loss: 0.7288 - val_accuracy: 0.2172
Epoch 5/100
125/125 [==============================] - 2s 14ms/step - loss: 0.0488 - accuracy: 0.2858 - val_loss: 0.7260 - val_accuracy: 0.2177
Epoch 6/100
125/125 [==============================] - 2s 13ms/step - loss: 0.0478 - accuracy: 0.2861 - val_loss: 0.7373 - val_accuracy: 0.2166
Epoch 7/100
125/125 [==============================] - 2s 14ms/step - loss: 0.0476 - accuracy: 0.2861 - val_loss: 0.7330 - val_accuracy:

125/125 [==============================] - 2s 14ms/step - loss: 0.0332 - accuracy: 0.2899 - val_loss: 0.8211 - val_accuracy: 0.2158
Epoch 58/100
125/125 [==============================] - 2s 14ms/step - loss: 0.0330 - accuracy: 0.2899 - val_loss: 0.8181 - val_accuracy: 0.2170
Epoch 59/100
125/125 [==============================] - 2s 14ms/step - loss: 0.0329 - accuracy: 0.2901 - val_loss: 0.8275 - val_accuracy: 0.2164
Epoch 60/100
125/125 [==============================] - 2s 18ms/step - loss: 0.0326 - accuracy: 0.2901 - val_loss: 0.8222 - val_accuracy: 0.2162
Epoch 61/100
125/125 [==============================] - 3s 28ms/step - loss: 0.0328 - accuracy: 0.2899 - val_loss: 0.8235 - val_accuracy: 0.2163
Epoch 62/100
125/125 [==============================] - 2s 14ms/step - loss: 0.0326 - accuracy: 0.2899 - val_loss: 0.8241 - val_accuracy: 0.2162
Epoch 63/100
125/125 [==============================] - 2s 15ms/step - loss: 0.0325 - accuracy: 0.2901 - val_loss: 0.8214 - val_accuracy: 0.216

In [ ]:

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [ ]:

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())



In [ ]:

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [ ]:


# for seq_index in range(100):
#     # Take one sequence (part of the training test)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print('-')
#     print('Input sentence:', input_texts[seq_index])
#     print('Decoded sentence:', decoded_sentence)